In [1]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import glob
import rosbag
import rospy
from urdf_parser_py.urdf import URDF
from pykdl_utils.kdl_parser import kdl_tree_from_urdf_model
from pykdl_utils.kdl_kinematics import KDLKinematics

In [2]:
data_path = './Robot_ViconOctober/'

In [3]:
# Extract Vicon and camera dynamic data
## Load Vicon data
vicobags_dyna = glob.glob(data_path+'ViconDynamicDataOct/*.bag')
for i,bag in enumerate(vicobags_dyna): # loop over vicon bags
    vdbag = rosbag.Bag(bag)
    vicon_dynaXpos = []
    vicon_dynaYpos = []
    vicon_dynaZpos = []
    viconbag_time_dyna = []
    for topicc, msgg, tt in vdbag.read_messages(topics=['/vicon/packbot_endEff/packbot_endEff']): # extract data
        vicon_dynaXpos.append(msgg.transform.translation.x)
        vicon_dynaYpos.append(msgg.transform.translation.y)
        vicon_dynaZpos.append(msgg.transform.translation.z) 
        vicon_time_dy = tt.secs
        viconbag_time_dyna.append(vicon_time_dy)
    vdbag.close()

    vicon_dynaXPosArr = np.array(vicon_dynaXpos)
    vicon_dynaYPosArr = np.array(vicon_dynaYpos)
    vicon_dynaZPosArr = np.array(vicon_dynaZpos)
    vicon_dynaPosArr = np.column_stack((vicon_dynaXPosArr,vicon_dynaYPosArr,vicon_dynaZPosArr)) 
    np.save(data_path+'vicon_dynaPosArr%d.npy'%i, vicon_dynaPosArr) # save to numpy
    np.save(data_path+'viconbag_time_dyna%d.npy'%i, viconbag_time_dyna)
    
# Load Robot data    
robobags_dyna = glob.glob(data_path+'RobotDynamicDataOct/*.bag')
for i,bag in enumerate(robobags_dyna): # loop over robot bags
    rdbag = rosbag.Bag(bag)
    robot_dynaXpos = []
    robot_dynaYpos = []
    robot_dynaZpos = []
    robotbag_Tagtime_dyna = [] # \this was spelt wrong
    for topi, msgs, tym in rdbag.read_messages(topics=['/tag_detections']): # extract data
        try:
            robot_dynaXpos.append(msgs.detections[0].pose.pose.pose.position.x)
            robot_dynaYpos.append(msgs.detections[0].pose.pose.pose.position.y)
            robot_dynaZpos.append(msgs.detections[0].pose.pose.pose.position.z)   
            robotbag_Tagtime_dyna.append(tym.secs)
        except:
            continue

    robot_dynaXposArr = np.array(robot_dynaXpos)
    robot_dynaYposArr = np.array(robot_dynaYpos)
    robot_dynaZposArr = np.array(robot_dynaZpos)
    robot_dynaposArr = np.column_stack((robot_dynaXposArr,robot_dynaYposArr,robot_dynaZposArr))
    np.save(data_path+'robot_dynaPosArr%d.npy'%i, robot_dynaposArr)
    np.save(data_path+'robotbag_Tagtime_dyna%d.npy'%i, robotbag_Tagtime_dyna)
    
    
    robotbag_Jointstime_dyna = []
    ja_list = []
    for topic2, msg2, t2 in rdbag.read_messages(topics=['/Packbot510/joints']):
        ja_list.append(msg2.position[:3])
        robotbag_Jointstime_dyna.append(t2.secs)
    ja_array_dyna = np.array(ja_list)
    rdbag.close()
    np.save(data_path+'ja_array_dyna%d.npy'%i, ja_array_dyna)
    np.save(data_path+'robotbag_Jointstime_dyna%d.npy'%i, robotbag_Jointstime_dyna)

In [4]:
# Sanity check loading data we just saved
vicon_dylist = []
camera_dylist = []
kinematics_list_dyna = []

tk = []
tc = []
tv = []

for i in range(len(vicobags_dyna)):
    vicon_dy = np.load(data_path+'vicon_dynaPosArr%d.npy'%i)
    vicon_dylist.append(vicon_dy)
    v_time = np.load(data_path+'viconbag_time_dyna%d.npy'%i) # Name was wrong. missing dyna
    tv.append(v_time)
   
    robot_dy = np.load(data_path+'robot_dynaPosArr%d.npy'%i)
    camera_dylist.append(robot_dy)
    c_time = np.load(data_path+'robotbag_Tagtime_dyna%d.npy'%i) # Name was wrong. missing dyna
    tc.append(c_time)
    
    RobotKin_dy = np.load(data_path+'ja_array_dyna%d.npy'%i)
    kinematics_list_dyna.append(RobotKin_dy)
    k_time = np.load(data_path+'robotbag_Jointstime_dyna%d.npy'%i)
    tk.append(k_time)
    

In [5]:
robot = URDF.from_xml_file('robot.urdf')
links = [rl.name for rl in robot.links]
kdl_kin_dy = KDLKinematics(robot,'base_link','elbow2_link')
#print(kdl_kin.get_joint_names())

BaseTo_ee_xyz_dyna = []
for dy in kinematics_list_dyna:
    ee_list_dy = []
    for joints in dy:
        posez_dy = kdl_kin_dy.forward(joints)
        ee_list_dy.append(posez_dy[0:3,3])
    BaseTo_ee_xyz_dyna.append(np.squeeze(np.array(ee_list_dy)))
np.save(data_path+'BaseTo_ee_xyz_dyna%d.npy'%i, BaseTo_ee_xyz_dyna)

In [6]:
def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform that maps corresponding points A to B in m spatial dimensions
    Input:
      A: Nxm numpy array of corresponding points
      B: Nxm numpy array of corresponding points
    Returns:
      T: (m+1)x(m+1) homogeneous transformation matrix that maps A on to B
      R: mxm rotation matrix
      t: mx1 translation vector
    '''
    
    #assert A.shape == B.shape  ####commented  all the assert statements 
                                ####coz they were gving me shape erros whereas the datasets shapes were the same,

    # get number of dimensions
    m = A.shape[1]

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    H = np.dot(AA.T, BB)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
       Vt[m-1,:] *= -1
       R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(m+1)
    T[:m, :m] = R
    T[:m, m] = t

    return T, R, t


def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nxm array of points
        dst: Nxm array of points
    Output:
        distances: Euclidean distances of the nearest neighbor
        indices: dst indices of the nearest neighbor
    '''
    
    #assert src.shape == dst.shape

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dst)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    return distances.ravel(), indices.ravel()


def icp(A, B, init_pose=None, max_iterations=20, tolerance=0.001):
    '''
    The Iterative Closest Point method: finds best-fit transform that maps points A on to points B
    Input:
        A: Nxm numpy array of source mD points
        B: Nxm numpy array of destination mD point
        init_pose: (m+1)x(m+1) homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation that maps A on to B
        distances: Euclidean distances (errors) of the nearest neighbor
        i: number of iterations to converge
    '''
    
    #assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # make points homogeneous, copy them to maintain the originals
    src = np.ones((m+1,A.shape[0]))
    dst = np.ones((m+1,B.shape[0]))
    src[:m,:] = np.copy(A.T)
    dst[:m,:] = np.copy(B.T)

    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0

    for i in range(max_iterations):
        # find the nearest neighbors between the current source and destination points
        distances, indices = nearest_neighbor(src[:m,:].T, dst[:m,:].T)

        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform(src[:m,:].T, dst[:m,indices].T)

        # update the current source
        src = np.dot(T, src)

        # check error
        mean_error = np.mean(distances)
        if np.abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error

    # calculate final transformation
    T,_,_ = best_fit_transform(A, src[:m,:].T)

    return T, distances #,i

In [7]:
TransMatrix, Dist = icp(BaseTo_ee_xyz_dyna[0],vicon_dylist[0], init_pose=None, max_iterations=20, tolerance=0.001)

In [8]:
# Rotate Robot data into Vicon frame of reference
robot_v_list = np.dot(TransMatrix,(BaseTo_ee_xyz_dyna[1]))

ValueError: shapes (4,4) and (1628,3) not aligned: 4 (dim 1) != 1628 (dim 0)

In [ ]:
# with the error above Im thinking of tryng one of the two below:
## 1. add a column of oness on the robot  data to have 4d xyz1 ? 
# or  2. use Rot  and Trans from best_fit_transform() like we did on the procrustes alignment previously?
##2a. But Im not sure how will 2. succeed if we ommit the work done by the two functions(nearest_neighbor()&icp())
# 2b. or will it make sense if myb I take the 4x4 TransMatrix and 
## treat TransMatrix[0:3,0:3] = 3*3 as Rot and then TransMatrixarr[3,0:3] as transla ?